In [1]:
import pandas as pd

import spacy
from langchain_community.retrievers import BM25Retriever
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.retrievers import EnsembleRetriever
import re
from typing import List

def extract_text_between(text):
    if not isinstance(text, str):
        return ''
    pattern = r'Krótki opis przedmiotu zamówienia(.*?)4\.2\.6\.\) Główny kod CPV'
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return ''

def extract_text_between_row(row):
    if not row['dokladny opis']:
        return row['opis']
    pattern = r'Krótki opis przedmiotu zamówienia(.*?)4\.2\.6\.\) Główny kod CPV'
    match = re.search(pattern, row['dokladny opis'], re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return row['opis']

nlp = spacy.load("pl_core_news_sm")
#drive.mount('/content/drive')

TOP_K = 10

/Users/krystianpietrzak/Documents/ML/Tender-AI/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Data Load

In [12]:
df = pd.read_csv('mnt/efs/data_pre_v2.csv')
df

,Unnamed: 0,Data Publikacji,Data Zakończenia,Miasto,Zamawiający,Przedmiot zamówienia,Link,Województwo,Krótki opis,Główny kod CPV,Unnamed: 9,processed_text,id,category,Województwo_proc,Województwo_proc_str,Category_proc,Category_proc_str
0,0,7/23/2024,2024-08-09,Lisów,ZESPÓŁ SZKOLNO -PRZEDSZKOLNY W LISOWIE,"""DOSTAWY ARTYKUŁÓW SPOŻYWCZYCH DO STOŁÓWKI ZE...",https://www.przetargi.egospodarka.pl/200426634...,śląskie,Pieczywo i wyroby piekarskie Mięso i wyroby m...,15811000-6 - Pieczywo 15110000-2 - Mięso 15600...,NaN,pieczywo wyrób piekarski mięso wyrób mięsny ...,0,artykuły spożywcze,['slaskie'],slaskie,['artykuły spożywcze'],artykuły spożywcze
1,1,7/23/2024,2024-07-31,Opole,UNIWERSYTET OPOLSKI,U/21/2024 Opracowanie aktualizacji PF-U dla in...,https://www.przetargi.egospodarka.pl/200426631...,opolskie,1.\tPrzedmiotem zamówienia jest: Opracowanie a...,71320000-7 - Usługi inżynieryjne w zakresie pr...,NaN,1 . \t przedmiot zamówienie : Opracowanie aktu...,1,budownictwo - wielobranżowe/specjalistyczne,['opolskie'],opolskie,['budownictwo - wielobranżowe/specjalistyczne'],budownictwo - wielobranżowe/specjalistyczne
2,2,7/23/2024,2024-08-08,Żnin,Gmina Żnin,"""Przebudowa ul. Kasztanowej w Żninie"".",https://www.przetargi.egospodarka.pl/200426628...,kujawsko-pomorskie,Planowana przebudowa drogi o nr 130028C ul. Ka...,45233220-7 - Roboty w zakresie nawierzchni dróg,NaN,planować przebudowo droga numer 130028C ulica ...,2,budownictwo - drogi chodniki place,['kujawsko-pomorskie'],kujawsko-pomorskie,['budownictwo - drogi chodniki place'],budownictwo - drogi chodniki place
3,3,7/23/2024,2024-07-31,Lwówek Śląski,ZARZĄD DRÓG POWIATOWYCH W LWÓWKU ŚLĄSKIM,Zimowe utrzymanie dróg powiatowych na terenie ...,https://www.przetargi.egospodarka.pl/200426624...,dolnośląskie,"CZĘŚĆ NR 1 – gmina Mirsk (długość ogółem: 51,5...",90620000-9 - Usługi odśnieżania 90620000-9 - U...,NaN,CZĘŚĆ NR 1 – gmina Mirsk ( długość ogóć być : ...,3,budownictwo - drogi chodniki place,['dolnoslaskie'],dolnoslaskie,['budownictwo - drogi chodniki place'],budownictwo - drogi chodniki place
4,4,7/23/2024,2024-07-31,Łańcut,MUZEUM-ZAMEK W ŁAŃCUCIE,"Usługa druku, oprawy oraz dostawy książki o ch...",https://www.przetargi.egospodarka.pl/200426617...,podkarpackie,"Przedmiotem zamówienia jest Usługa druk, opraw...",79823000-9 - Usługi drukowania i dostawy,NaN,"przedmiot zamówienie usługa druk , oprawa dost...",4,poligrafia,['podkarpackie'],podkarpackie,['poligrafia'],poligrafia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11412,11412,07/19/2024,07/30/2024,oławski,ZARZĄD POWIATU W OŁAWIE,Świadczenie usług agenta emisji obligacji,https://www.oferent.com.pl/przetargi/swiadczen...,Dolnośląskie,Świadczenie usług Agenta emisji obligacji Powi...,finanse rachunkowośćemisje obligacji,NaN,świadczenie usługa Agent emisja obligacja powi...,11412,NaN,['dolnoslaskie'],dolnoslaskie,"['administrowanie obiektami/lokalami', 'artyku...","administrowanie obiektami/lokalami, artykuły b..."
11413,11413,07/19/2024,08/05/2024,Wrocław,Wrocławski Teatr Współczesny,Wymiana dźwigu osobowego w budynku,https://www.oferent.com.pl/przetargi/wymiana-d...,Dolnośląskie,Wymiana dźwigu osobowego w budynku Wrocławskie...,"windy i dźwigiwindy osobowe, towarowe, schodoł...",NaN,wymiana dźwig osobowy budynek Wrocławski teatr...,11413,NaN,['dolnoslaskie'],dolnoslaskie,"['administrowanie obiektami/lokalami', 'artyku...","administrowanie obiektami/lokalami, artykuły b..."
11414,11414,07/19/2024,07/31/2024,Poznań,Poznańskie Ośrodki Sportu i Rekreacji,Gadżety,"https://www.oferent.com.pl/przetargi/gadzety,2...",Wielkopolskie,Gadżety dla POSiR oraz organizowanych przez PO...,artykuły biurowe osprzęt biurowyinne biurowear...,NaN,Gadżety POSiR organizować POSiR impreza . 1 . ...,11414,NaN,['wielkopolskie'],wielkopolskie,"['administrowanie obiektami/lokalami', 'artyku...","administrowanie obiektami/lokalami, artykuły b..."
11415,11415,07/19/2024,08/01/2024,oleśnicki,"Oleśnicki Kompleks Rekreacyjny ""ATOL""

# Sparse retriever training

In [147]:
def preprocess(text):
    if isinstance(text, str):
      doc = nlp(text)
      return " ".join([token.lemma_ for token in doc if not token.is_stop])
    else:
      return ''

In [148]:
df['processed_text'] = df['Krótki opis'].apply(preprocess)

In [24]:
df.to_csv('mnt/efs/data_pre_v2.csv')

In [151]:
ids = [i for i in range(len(df))]
df['id'] =ids

In [13]:
df['category'] = df['kategoria - colab']
df = df.drop(columns='kategoria - colab')

In [16]:
df['category'].unique()

array(['artykuły spożywcze',
       'budownictwo - wielobranżowe/specjalistyczne',
       'budownictwo - drogi chodniki place', 'poligrafia',
       'sprzątanie pielęgnacja - obiektowa',
       'szkolenia edukacja tłumaczenia', 'usługi społeczne',
       'sprzęt szkolny dydaktyczny sportowy',
       'ochrona osób mienia konwojowanie',
       'budownictwo - obiekty sportowe',
       'oświetlenie elektryczne artykuły', 'zabytki konserwacje',
       'komputery informatyka internet',
       'utylizacja odpadów oczyszczalnie',
       'materiały prefabrykaty budowlane', 'gastronomia',
       'transport przewozy', 'elektrotechniczne urządzenia',
       'meble/wyposażenie wnętrz', 'motoryzacja',
       'medycyna - sprzęt aparatura', 'ubezpieczenia',
       'geodezja geologia kartografia urbanistyka',
       'turystyka imprezy masowe', 'zagospodarowanie terenu',
       'budownictwo - wodno-kanalizacyjne', 'nieruchomości',
       'laboratoria - osprzęt usługi', 'medycyna - leki opatrunki',
     

In [ ]:
budownictwo - obiekty wykończeniabudownictwo - obsługabudownictwo - wielobranżowe/specjalistyczne

In [17]:
df.to_csv('mnt/efs/data_pre_v2.csv', index=False)

In [3]:
df = pd.read_csv('mnt/efs/data_pre_v2.csv')

In [9]:
corpus = df['processed_text'].tolist()
corpus = [str(text) for text in corpus]
metadata = [{'id': id} for id in df['id'].tolist()]

bm25_retriever = BM25Retriever.from_texts(corpus, metadatas=metadata)
bm25_retriever.k = TOP_K

In [7]:
# Example usage
query = '''
MarkX	Piotr Rajshttps://www.markx-rajs.pl/	budownictow drogowe; Drogi, nasypy stabilizacje kanalizacje, ale tez duzo innych- patrz strona. Lokalizacja: cala Polska
'''

def preprocess_(text):
    text = text.replace('\n', ' ').replace('\t', ' ')
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return tokens

bm25_retriever.preprocess_func = preprocess_
results = bm25_retriever.invoke(query)

print("Query:", query)
results[0]

[' ', 'markx', 'piotr', 'rajshttps://www.markx-rajs.pl/', 'budownictow', 'drogowy', 'droga', 'nasypa', 'stabilizacja', 'kanalizacja', 'tez', 'patrzyć', 'strona', 'lokalizacja', 'polska']
Query: 
MarkX	Piotr Rajshttps://www.markx-rajs.pl/	budownictow drogowe; Drogi, nasypy stabilizacje kanalizacje, ale tez duzo innych- patrz strona. Lokalizacja: cala Polska



Document(metadata={'id': 1985}, page_content='przedmiot zamówienie robota budowlany polegać budowa ulica . zaciszy Darłów zgodnie wydany decyzja pozwolenie budowa Nr 53/2022 dzień 1907 . 02 . 2022 rok . znak BS . 6740 . 846 . 2021 . załączyć dokumentacja projektowy . roboty budowlany polegać wykonać robota poszczególny branż szczególność : . \t branża sanitarnej: - budowa kanalizacja deszczowy , wykonać wpuset drogowy , przykanalik granica działka , - włączyć nowy - wybudowany sieć istniejący sieć kanalizacja deszczowy ( rurociąg DN1000), - montaż osadnik separator piasek wraz klapa zwrotną; - usunąć kolizja istniejący sieci przyłączami.b . \t branża elektrycznej: - demontaż istnieć oświetlenie ulica unieczynnieć istnieć kabel oświetleniowego; - budowa nowy oświetlenie poprzez rozwinięt sieć elektroenergetyczny montaż 8 szt . słup oświetlenie ( słupa oprawa demontażu); - usunięcić kolizja istniejący sieci niskiy średni napięcia;c . \t branża drogowej: - wykonać niezbędny rozbiórka istn

# Dense retriever training

In [8]:
metadata = [{'id': id} for id in df['id'].tolist()]

texts = df['Krótki opis'].to_list()
texts = [str(text) for text in texts]

embedding_function = HuggingFaceEmbeddings(model_name="mnt/efs/model")

chromadb_index = Chroma.from_texts(
    texts, embedding_function, persist_directory='mnt/efs/vectorDB', metadatas=metadata
)

KeyboardInterrupt: 

# Ensemble

In [15]:
embedding_function = HuggingFaceEmbeddings(model_name="mnt/efs/model")
chromadb_index = Chroma(persist_directory='mnt/efs/vectorDB', embedding_function=embedding_function)


/Users/krystianpietrzak/Documents/ML/Tender-AI/.venv/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/krystianpietrzak/Documents/ML/Tender-AI/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [42]:
index = chromadb_index._collection.peek(chromadb_index.__len__())

In [46]:
ids = [id['id'] for id in index['metadatas']]

In [36]:
chromadb_index.get(where={"id": 11416})

{'ids': ['2ba5013d-73d4-4a02-b37f-1c1c22766719'],
 'embeddings': None,
 'metadatas': [{'id': 11416}],
 'documents': ['Wykonywanie usług sprzątania w obiekcie będącym siedzibą WFOŚiGW we Wrocławiu, Oddział Biura w Jeleniej Górze zlokalizowanym przy Ul. Ratuszowej 31/32 w Jeleniej Górze.Krótki opis przedmiotu zamówieniaPrzedmiotem zamówienia jest wykonywanie usług sprzątania w obiekcie będącym siedzibą WFOŚiGW we Wrocławiu, Oddział Biura w Jeleniej Górze zlokalizowanym przy Ul. Ratuszowej 31/32 w Jeleniej Górze. Szczegółowy opis przedmiotu zamówienia zawiera załącznik nr 1 do SWZ.'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [5]:
chroma_retriever = chromadb_index.as_retriever(search_kwargs={"k": TOP_K, 'filter':{'id':{'$in':[0,1,2,3,4,5]}}})
query = '''
MarkX	Piotr Rajshttps://www.markx-rajs.pl/	budownictow drogowe; Drogi, nasypy stabilizacje kanalizacje, ale tez duzo innych- patrz strona. Lokalizacja: cala polska
'''
chroma_retriever.get_relevant_documents(query)

/Users/krystianpietrzak/Documents/ML/Tender-AI/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'id': 2}, page_content='Planowana przebudowa drogi o nr 130028C ul. Kasztanowa w Żninie posiada nawierzchnię gruntową, która w obecnym stanie technicznym uniemożliwia bezpieczne korzystanie z przedmiotowej drogi. Z uwagi na poprawę bezpieczeństwa uczestników ruchu drogowego konieczną staje się również przebudowa infrastruktury oświetleniowej oraz budowa chodnika.- klasa techniczna drogi (ulicy) „D” – dojazdowa- droga jednojezdniowa dwupasowa- numer drogi publicznej 131028C- długość drogi 511,85 m- szerokość jezdni 4,50 m- szerokość pasa ruchu 2,25m- szerokość utwardzonego pobocza 1*0,75 m- spadek poprzeczny jezdni jednostronny 2%- szerokość jednostronnego chodnika z kostki betonowej 2,00 m- spadek poprzeczny pobocza jednostronny 6%- skrajnia pionowa 4,50m'),
 Document(metadata={'id': 3}, page_content='CZĘŚĆ NR 1 – gmina Mirsk (długość ogółem: 51,517 km):Nr drogi\tPrzebieg\tDługość odcinków\tStandard do odśnieżania w km\tdo posypywania w km1.\t2492D\tKwieciszowice-gr

In [6]:
ALFA = 0.7
CHROMA_WEIGHT = ALFA
BM25_WEIGHT = 1 - ALFA
TOP_K = 20

In [10]:
cat = ["budownictwo - drogi chodniki place",
       "budownictwo - wodno-kanalizacyjne",
       "budownictwo - sanitarno-hydrauliczne",]

metadata = [{'id': id} for id in df['id'].tolist()]

corpus = df['processed_text'].tolist()
corpus = [str(text) for text in corpus]

search_ids = df[df['kategoria - colab'].isin(cat)]['id'].tolist()

search_corp = []
search_meta = []
for id, corp in zip(metadata, corpus):
    if id['id'] in search_ids:
        search_corp.append(corp)
        search_meta.append({'id':id['id']})

In [11]:
def preprocess_(text: str) -> List[str]:
    doc = nlp(text)
    return [token.lemma_ for token in doc if not token.is_stop]

chroma_retriever = chromadb_index.as_retriever(search_kwargs={"k": TOP_K, 'filter':{'id':{'$in':search_ids}}})

bm25_retriever = BM25Retriever.from_texts(search_corp, metadatas=search_meta)
bm25_retriever.preprocess_func = preprocess_
bm25_retriever.k = TOP_K

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[BM25_WEIGHT, CHROMA_WEIGHT]
)

query = '''
MarkX	Piotr Rajshttps://www.markx-rajs.pl/	budownictow drogowe; Drogi, nasypy stabilizacje kanalizacje, ale tez duzo innych- patrz strona. Lokalizacja: cala polska
'''

query_extended = '''
W Markx zajmujemy się przede wszystkim realizacją prac ziemnych i budowlanych w obrębie dróg oraz autostrad. 
Współpracę podejmujemy z różnego rodzaju przedsiębiorstwami, które poszukują zaufanego kontrahenta. 
Ponadto wykonujemy także roboty dodatkowe, spośród których wymienić można działania mające na celu:
- stabilizację i wzmocnienie gruntów,
- wykonanie przyłączy wodociągowych,
- budowę sieci gazowych,
- budowę zbiorników retencyjnych.
Ponadto zajmujemy się też montażem przepompowni ścieków zarówno mniejszych (np. przy gospodarstwie rolnym lub domowym), jak i większych (w przypadku dużych hal i rozległych zakładów przemysłowych). 
W przypadku wszystkich zleceń dot. budownictwa drogowego oraz innych prac zapewniamy jasne warunki ofertowe, profesjonalne wsparcie oraz doradztwo na każdym etapie współpracy. 
'''

docs = ensemble_retriever.invoke(query_extended)

docs

['\n', 'Markx', 'zajmować', 'realizacja', 'praca', 'ziemny', 'budowlany', 'obręb', 'droga', 'autostrada', '.', '\n', 'Współpraca', 'podejmować', 'różny', 'rodzaj', 'przedsiębiorstwo', ',', 'poszukywać', 'zaufany', 'kontrahent', '.', '\n', 'ponadto', 'wykonywać', 'robota', 'dodatkowy', ',', 'spośród', 'wymienić', 'działanie', 'mieć', 'cel', ':', '\n', '-', 'stabilizacja', 'wzmocnić', 'grunt', ',', '\n', '-', 'wykonać', 'przyłącz', 'wodociągowy', ',', '\n', '-', 'budowa', 'sieć', 'gazowy', ',', '\n', '-', 'budowa', 'zbiorniek', 'retencyjny', '.', '\n', 'ponadto', 'zajmujemy', 'montaż', 'przepompownia', 'ściek', 'zarówno', 'mniejszych', '(', 'na przykład', '.', 'gospodarstwo', 'rolny', 'domowy', ')', ',', 'duży', '(', 'przypadek', 'duży', 'hala', 'rozległy', 'zakład', 'przemysłowy', ')', '.', '\n', 'przypadek', 'zlecenie', 'dot', '.', 'budownictwo', 'drogowy', 'praca', 'zapewniać', 'jasny', 'warunek', 'ofertowy', ',', 'profesjonalny', 'wsparcie', 'doradztwo', 'każdy', 'etap', 'współpraca'

[Document(metadata={'id': 578}, page_content='Przedmiotem zamówienia są roboty budowlane:1. Wykonanie robót przygotowawczych:a)\tRoboty pomiarowe przy liniowych robotach ziemnych;b)\tUsunięcie warstwy ziemi urodzajnej (humusu) o grubości do 15 cm;c)\tWykonanie frezowania korekcyjnego nawierzchni bitumicznych;2. Wykonanie nakładki na nawierzchni z mieszanek mineralno-bitumicznych dróg i placów:a)\tMechaniczna rozbiórka krawężników betonowych z ławą betonową wraz z wywozem (mechaniczne cięcie nawierzchni bitumicznych);b)\tUstawienie krawężników betonowych na ławach z betonu C12/15 (stojących, zatopionych i leżących);c)\tRegulacja studzienek, włazów, zaworów;d)\tMechaniczne oczyszczenie nawierzchni asfaltowych;e)\tWykonanie podbudowy z kruszywa łamanego stabilizowanego mechanicznie 0/31,5 gr. 10 cm;f)\tOczyszczenie i skropienie podbudowy pomocniczej bitumem 0,7kg/m2;g)\tWykonanie warstwy wiążącej z betonu asfaltowego AC16W 50/70 gr. 5 cm;h)\tOczyszczenie i skropienie warstwy wiążącej bitu

In [1]:
import pandas as pd
from utils.llm import LLM

llm = LLM()

query_df= pd.read_csv('mnt/efs/data_query1.csv')

In [67]:
query_df.head(10)

,Firma,Imie i nazwisko,Tel,email,www,data kontkatu,Free raport sent?,typy dzialalnosci,obszar działalności,feedback call,paying?,notes,kategorie colab,enhanced opis gpt\n(na podstawie strony)
0,Lenpol,Krzysztof Czepukowicz,605895476,info@lenpol.pl,lenpol.pl,15/07/2024,NaN,"wyburzenia, rozbiórki",lubuskie,17/07 - nie odbiera,NaN,NaN,rozbiórki wyburzenia,Firma specjalizuje się w wyburzeniach i rozbió...
1,Nowy Dach FHU,Roman Mejer,511016044,nowydach27@wp.pl,http://www.nowy-dach.com.pl/,15/07/2024,NaN,"pokrycia dachowe, hurtownia",pomorski,blisko nas szkoły itd. bez procedury nieraz si...,NaN,NaN,dekartstwo\nmateriały prefabrykaty budowlane,"Firma z siedzibą w Nowej Karczmie, specjalizuj..."
2,Gut-Bud,Magdalena,606 663 308,gut-bud@wp.pl,https://gut-bud.pl/,15/07/2024,NaN,"elewacje, budowy domów",mazowieckie,17/07 - nie odbiera,NaN,NaN,budownictwo - obiekty wykończenia\nbudownictwo...,Specjalizuje się w budowie domów oraz pracach ...
3,Grinbud,Dominik Sieszchuła,663 007 302,biuro@grinbud.pl,https://www.grinbud.pl/,15/07/2024,NaN,roboty budowlane,"wielkopolska, kujawsko pomorskie",16/07 - on ma dostep do biznespolska i tam wsz...,not interested,NaN,budownictwo - wielobranżowe/specjalistyczne\n,Firma zajmuje się szerokim zakresem robót budo...
4,Montplast,Iwona,696 492 894,biuro@montplast.pl,https://www.montplast.com.pl/,15/07/2024,NaN,tylko wymiana okien i drzwi,łódzkie,pani jest tylko od tego i analizuje maja bizne...,NaN,"Iwona - jedzie na urlop nad moze, od piatku je...",okna/drzwi/bramy/ogrodzenia\n\n\n\n\n\n,Specjalizuje się w wymianie okien i drzwi. Fir...
5,Jati,Jacek Kaczor,503903511,jatimeble@gmail.com,https://jati.pl/,12/07/2024,NaN,"lady recepcyjne, hotele, stolarka na wymiar","małopolska, cala polska",wyslac wiecej,NaN,NaN,Meble/wyposażenie wnętrz\nStolarka/drewno\nPro...,Jati Sp. z o.o. specjalizuje się w kompleksowy...
6,EkoWodrol,Adrianna Wiśniewska - przetargi,693580282,adrianna.wisniewska@ekowodrol.eu,https://ekowodrol.pl/,12/07/2024,NaN,sieci wodociagowo kanalizacyjne,"kujawsko pomorskie, pomorskie, lubuskie, wielk...",nieodbiera - 17/07,NaN,NaN,Budownictwo - wodno-kanalizacyjne\nAutomatyka ...,EkoWodrol Sp. z o.o. specjalizuje się w komple...
7,Jadar SC,"Jarosław Niedzielski , Adam Balicki",502 179 911,jadarsc@gmail.com,http://www.jadar.eu/,12/07/2024,NaN,sieci wodno kanalziacyjne na koparki,tylko w obrebie 70km od jelenie gory,yes,yes,NaN,Budownictwo - sanitarno-hydrauliczne\nKlimatyz...,Instalacje Iwanicki specjalizuje się w instala...
8,ISO Iwanicki,Paweł Iwanicki,602 559 627,instalacje@op.pl,https://www.instalacjeiwanicki.pl/,12/07/2024,NaN,instalacje hale wieskze miekszje kanalizacja,NaN,ma zobaczyc dzisiaj - 16/07,NaN,NaN,Budownictwo - wodno-kanalizacyjne\nUsługi sprz...,Jadar specjalizuje się w budowie sieci wodno-k...
9,Wodner,Kazimierz Żurawski,607 994 355,wodner@interia.pl,NaN,12/07/2024,NaN,NaN,łódzkie,ma rzucic okiem - przycisnij -16/07,NaN,NaN,NaN,NaN


In [128]:
for i in range(10):
    url = query_df.iloc[i]['enhanced opis gpt\n(na podstawie strony)']
    print(url)

Firma specjalizuje się w wyburzeniach i rozbiórkach budynków. Ich oferta obejmuje kompleksowe usługi związane z demontażem i usuwaniem konstrukcji budowlanych. Poszukują przetargów związanych z wyburzeniami budynków, usuwaniem gruzu oraz przygotowaniem terenu pod nowe inwestycje.
Firma z siedzibą w Nowej Karczmie, specjalizująca się w usługach dekarskich na terenie województwa pomorskiego. Oferują szeroki wachlarz usług, w tym budowę dachów, remonty, sprzedaż materiałów dekarskich (np. blachodachówki, dachówki, gonty bitumiczne) oraz montaż orynnowania i okien dachowych. Firma poszukuje przetargów na budowę i remonty dachów, a także na dostawy materiałów budowlanych związanych z pokryciami dachowymi​​​​​​.


Specjalizuje się w budowie domów oraz pracach elewacyjnych. Firma oferuje kompleksowe usługi budowlane, w tym wznoszenie budynków mieszkalnych i komercyjnych, jak również modernizacje elewacji. Gut-Bud szuka przetargów na budowę nowych domów, remonty i renowacje elewacji oraz inne 

In [56]:
query_df['obszar działalności'].unique()

array(['lubuskie', 'pomorski', 'mazowieckie',
       'wielkopolska, kujawsko pomorskie', 'łódzkie',
       'małopolska, cala polska',
       'kujawsko pomorskie, pomorskie, lubuskie, wielkopolskie zach pomorskie  250 km',
       'tylko w obrebie 70km od jelenie gory', nan, 'Polska',
       'świętokrzyskie - Radków', '100 km od radomia', 'pomorskie ',
       '100km max bielski żywiecki oświęcimski 100km max ',
       'max 30km od Krakowa', 'lubuskie, dolnośląskie',
       'najlepiej wielkopolska ale moze byc dalej',
       'zielonogorski swiebodzinski ', 'małopolskie, śląskie',
       'tylko łódzkie ', 'wielkopolskie', 'max katowice', 'cala polska',
       'zach-pomorskie'], dtype=object)

In [73]:
# Mapping of special Polish characters to their regular equivalents
polish_to_regular = str.maketrans(
    'ąćęłńóśźż', 
    'acelnoszz'
)

# List of Polish voivodeships
voivodeships = [
    'slaskie', 'opolskie', 'kujawsko-pomorskie', 'dolnoslaskie', 
    'podkarpackie', 'warminsko-mazurskie', 'wielkopolskie', 'pomorskie',
    'lubelskie', 'lodzkie', 'mazowieckie', 'zachodniopomorskie',
    'malopolskie', 'podlaskie', 'swietokrzyskie', 'lubuskie'
]

def process_wojewodztwo(woj):
    if pd.isna(woj):
        return voivodeships
    # Remove numbers and parentheses
    woj = re.sub(r'\(\d+\)', '', woj)
    
    woj = woj.lower().strip().replace('pokaż dodatkowe','')
    
    # Replace special Polish characters
    woj = woj.translate(polish_to_regular)

    # Split concatenated voivodeships
    result = []
    temp = woj
    for v in voivodeships:
        temp = temp.replace(v, v + ',')
    result = temp.strip(',').split(',')
    
    return [v for v in result if v]

# Apply the function to the DataFrame
df['Województwo_proc'] = df['Województwo'].apply(process_wojewodztwo)

# Convert lists to strings to find unique values
df['Województwo_proc_str'] = df['Województwo_proc'].apply(lambda x: ', '.join(sorted(x)))

# Get unique string representations
unique_values = df['Województwo_proc_str'].unique()

# Display unique values
for value in unique_values:
    print(value)

slaskie
opolskie
kujawsko-pomorskie
dolnoslaskie
podkarpackie
warminsko-mazurskie
wielkopolskie
pomorskie
lubelskie
lodzkie
mazowieckie
zachodniopomorskie
malopolskie
podlaskie
swietokrzyskie
lubuskie
dolnoslaskie, kujawsko-pomorskie, lodzkie, lubelskie, lubuskie, malopolskie, mazowieckie, opolskie, podkarpackie, podlaskie, pomorskie, slaskie, swietokrzyskie, warminsko-mazurskie, wielkopolskie, zachodniopomorskie
lubelskie, warminsko-mazurskie
kujawsko-pomorskie, swietokrzyskie
kujawsko-pomorskie, pomorskie
mazowieckie, podkarpackie
pomorskie, wielkopolskie
mazowieckie, podkarpackie, wielkopolskie
malopolskie, opolskie, swietokrzyskie
dolnoslaskie, mazowieckie
lubuskie, wielkopolskie
mazowieckie, podlaskie
lubuskie, mazowieckie
mazowieckie, pomorskie
lodzkie, lubelskie, mazowieckie, wielkopolskie
slaskie, wielkopolskie
opolskie, slaskie
lodzkie, mazowieckie
wielkopolskie, zachodniopomorskie
dolnoslaskie, malopolskie
dolnoslaskie, lubuskie
mazowieckie, warminsko-mazurskie
lubelskie, maz

In [10]:
df.to_csv('mnt/efs/data_pre_v2.csv', index=False)

In [12]:
df

,Unnamed: 0,Data Publikacji,Data Zakończenia,Miasto,Zamawiający,Przedmiot zamówienia,Link,Województwo,Krótki opis,Główny kod CPV,Unnamed: 9,processed_text,id,category,Województwo_proc,Województwo_proc_str,Category_proc,Category_proc_str
0,0,7/23/2024,2024-08-09,Lisów,ZESPÓŁ SZKOLNO -PRZEDSZKOLNY W LISOWIE,"""DOSTAWY ARTYKUŁÓW SPOŻYWCZYCH DO STOŁÓWKI ZE...",https://www.przetargi.egospodarka.pl/200426634...,śląskie,Pieczywo i wyroby piekarskie Mięso i wyroby m...,15811000-6 - Pieczywo 15110000-2 - Mięso 15600...,NaN,pieczywo wyrób piekarski mięso wyrób mięsny ...,0,artykuły spożywcze,['slaskie'],slaskie,[artykuły spożywcze],artykuły spożywcze
1,1,7/23/2024,2024-07-31,Opole,UNIWERSYTET OPOLSKI,U/21/2024 Opracowanie aktualizacji PF-U dla in...,https://www.przetargi.egospodarka.pl/200426631...,opolskie,1.\tPrzedmiotem zamówienia jest: Opracowanie a...,71320000-7 - Usługi inżynieryjne w zakresie pr...,NaN,1 . \t przedmiot zamówienie : Opracowanie aktu...,1,budownictwo - wielobranżowe/specjalistyczne,['opolskie'],opolskie,[budownictwo - wielobranżowe/specjalistyczne],budownictwo - wielobranżowe/specjalistyczne
2,2,7/23/2024,2024-08-08,Żnin,Gmina Żnin,"""Przebudowa ul. Kasztanowej w Żninie"".",https://www.przetargi.egospodarka.pl/200426628...,kujawsko-pomorskie,Planowana przebudowa drogi o nr 130028C ul. Ka...,45233220-7 - Roboty w zakresie nawierzchni dróg,NaN,planować przebudowo droga numer 130028C ulica ...,2,budownictwo - drogi chodniki place,['kujawsko-pomorskie'],kujawsko-pomorskie,[budownictwo - drogi chodniki place],budownictwo - drogi chodniki place
3,3,7/23/2024,2024-07-31,Lwówek Śląski,ZARZĄD DRÓG POWIATOWYCH W LWÓWKU ŚLĄSKIM,Zimowe utrzymanie dróg powiatowych na terenie ...,https://www.przetargi.egospodarka.pl/200426624...,dolnośląskie,"CZĘŚĆ NR 1 – gmina Mirsk (długość ogółem: 51,5...",90620000-9 - Usługi odśnieżania 90620000-9 - U...,NaN,CZĘŚĆ NR 1 – gmina Mirsk ( długość ogóć być : ...,3,budownictwo - drogi chodniki place,['dolnoslaskie'],dolnoslaskie,[budownictwo - drogi chodniki place],budownictwo - drogi chodniki place
4,4,7/23/2024,2024-07-31,Łańcut,MUZEUM-ZAMEK W ŁAŃCUCIE,"Usługa druku, oprawy oraz dostawy książki o ch...",https://www.przetargi.egospodarka.pl/200426617...,podkarpackie,"Przedmiotem zamówienia jest Usługa druk, opraw...",79823000-9 - Usługi drukowania i dostawy,NaN,"przedmiot zamówienie usługa druk , oprawa dost...",4,poligrafia,['podkarpackie'],podkarpackie,[poligrafia],poligrafia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11412,11412,07/19/2024,07/30/2024,oławski,ZARZĄD POWIATU W OŁAWIE,Świadczenie usług agenta emisji obligacji,https://www.oferent.com.pl/przetargi/swiadczen...,Dolnośląskie,Świadczenie usług Agenta emisji obligacji Powi...,finanse rachunkowośćemisje obligacji,NaN,świadczenie usługa Agent emisja obligacja powi...,11412,NaN,['dolnoslaskie'],dolnoslaskie,"[administrowanie obiektami/lokalami, artykuły ...","administrowanie obiektami/lokalami, artykuły b..."
11413,11413,07/19/2024,08/05/2024,Wrocław,Wrocławski Teatr Współczesny,Wymiana dźwigu osobowego w budynku,https://www.oferent.com.pl/przetargi/wymiana-d...,Dolnośląskie,Wymiana dźwigu osobowego w budynku Wrocławskie...,"windy i dźwigiwindy osobowe, towarowe, schodoł...",NaN,wymiana dźwig osobowy budynek Wrocławski teatr...,11413,NaN,['dolnoslaskie'],dolnoslaskie,"[administrowanie obiektami/lokalami, artykuły ...","administrowanie obiektami/lokalami, artykuły b..."
11414,11414,07/19/2024,07/31/2024,Poznań,Poznańskie Ośrodki Sportu i Rekreacji,Gadżety,"https://www.oferent.com.pl/przetargi/gadzety,2...",Wielkopolskie,Gadżety dla POSiR oraz organizowanych przez PO...,artykuły biurowe osprzęt biurowyinne biurowear...,NaN,Gadżety POSiR organizować POSiR impreza . 1 . ...,11414,NaN,['wielkopolskie'],wielkopolskie,"[administrowanie obiektami/lokalami, artykuły ...","administrowanie obiektami/lokalami, artykuły b..."
11415,11415,07/19/2024,08/01/2024,oleśnicki,"Oleśnicki Kompleks Rekreacyjny ""ATOL"" Sp. z o.o

In [8]:
# List of all available categories
categories = [
    'administrowanie obiektami/lokalami', 'artykuły biurowe osprzęt biurowy',
    'artykuły gospodarstwa domowego', 'artykuły spożywcze', 'automatyka elektronika',
    'bankowość kredyty', 'bhp bezpieczeństwo pracy', 'budownictwo - ciepłownictwo',
    'budownictwo - drogi chodniki place', 'budownictwo - mosty wiadukty', 
    'budownictwo - obiekty sportowe', 'budownictwo - obiekty wykończenia', 
    'budownictwo - obsługa', 'budownictwo - sanitarno-hydrauliczne', 
    'budownictwo - wielobranżowe/specjalistyczne', 'budownictwo - wodno-kanalizacyjne',
    'dekartstwo', 'długoterminowe analizy', 'elektrotechniczne urządzenia', 
    'elektryka energetyka osprzęt', 'elementy bezpieczeństwa ruchu drogowego', 
    'finanse rachunkowość', 'firmy, przeniesienie własności', 'gastronomia', 
    'gazownictwo', 'geodezja geologia kartografia urbanistyka', 'górnictwo - roboty maszyny', 
    'gospodarka leśna', 'klimatyzacja wentylacja chłodnictwo', 'kolej - roboty kolejowe', 
    'kolej - wagony maszyny sprzęt', 'komputery informatyka internet', 
    'konsolidacja, fuzje, przejęcia, rozwój', 'kupony bony towarowe dodatki', 
    'laboratoria - osprzęt usługi', 'maszyny/urządzenia/mechanika', 
    'materiały prefabrykaty budowlane', 'meble/wyposażenie wnętrz', 
    'medycyna - laboratoria środki chemiczne', 'medycyna - leki opatrunki', 
    'medycyna - sprzęt aparatura', 'medycyna usługi', 'melioracje regulacje rzek', 
    'metalurgia wyroby hutnicze', 'motoryzacja', 'motoryzacja - części, urządzenia, usługi', 
    'motoryzacja - specjalistyczna', 'nadzór doradztwo obsługa przedsiębiorstw', 
    'nieruchomości', 'niwelacje terenu roboty ziemne', 'ochrona osób mienia konwojowanie', 
    'ocieplanie budynków termomodernizacje', 'odzież tekstylia', 'ogrodnictwo sadownictwo', 
    'okna/drzwi/bramy/ogrodzenia', 'oświetlenie elektryczne artykuły', 'papier folie opakowania', 
    'poligrafia', 'projektowanie/dokumentacja/architektura', 'przechowywanie składowanie', 
    'reklama prasa media', 'rolnictwo/agrotechnika', 'rozbiórki wyburzenia', 
    'ślusarstwo obróbka metali', 'sprzątanie pielęgnacja - obiektowa', 
    'sprzątanie pielęgnacja - terenowa', 'sprzęt pływający statki okręty', 
    'sprzęt szkolny dydaktyczny sportowy', 'środki chemiczne tworzywa sztuczne', 
    'środki czystości i higieny', 'środowisko naturalne ekologia', 'statki powietrzne i wyposażenie', 
    'stolarka/drewno', 'surowce paliwa opał', 'systemy alarmowe i p.poż.', 'szkolenia edukacja tłumaczenia', 
    'telekomunikacja łączność', 'transport przewozy', 'turystyka imprezy masowe', 'ubezpieczenia', 
    'usługi artykuły drobnego handlu', 'usługi kamieniarskie', 'usługi pracy tymczasowej', 
    'usługi społeczne', 'usługi sprzętem ciężkim', 'utylizacja odpadów oczyszczalnie', 
    'windy i dźwigi', 'wojskowość/militaria/policja', 'zabytki konserwacje', 
    'zagadnienia niesklasyfikowane', 'zagospodarowanie terenu'
]

def process_categories(text):
    if pd.isna(text):
        return categories
    result = []
    temp = text
    for category in categories:
        temp = temp.replace(category, category + '|')
    result = temp.strip('|').split('|')
    return [v.strip() for v in result if v]

# Apply the function to the DataFrame
df['Category_proc'] = df['category'].apply(process_categories)

# Convert lists to strings to find unique values
df['Category_proc_str'] = df['Category_proc'].apply(lambda x: ', '.join(sorted(x)))

# Get unique string representations
unique_values = df['Category_proc_str'].unique()

# Display unique values
for value in unique_values:
    print(value)


artykuły spożywcze
budownictwo - wielobranżowe/specjalistyczne
budownictwo - drogi chodniki place
poligrafia
sprzątanie pielęgnacja - obiektowa
szkolenia edukacja tłumaczenia
usługi społeczne
sprzęt szkolny dydaktyczny sportowy
ochrona osób mienia konwojowanie
budownictwo - obiekty sportowe
oświetlenie elektryczne artykuły
zabytki konserwacje
komputery informatyka internet
utylizacja odpadów oczyszczalnie
materiały prefabrykaty budowlane
gastronomia
transport przewozy
elektrotechniczne urządzenia
meble/wyposażenie wnętrz
motoryzacja
medycyna - sprzęt aparatura
ubezpieczenia
geodezja geologia kartografia urbanistyka
turystyka imprezy masowe
zagospodarowanie terenu
budownictwo - wodno-kanalizacyjne
nieruchomości
laboratoria - osprzęt usługi
medycyna - leki opatrunki
projektowanie/dokumentacja/architektura
reklama prasa media
melioracje regulacje rzek
artykuły biurowe osprzęt biurowy
budownictwo - obiekty wykończenia
medycyna usługi
gospodarka leśna
maszyny/urządzenia/mechanika
usługi art

In [8]:
fill_list = df['Województwo'].unique().tolist()

In [9]:
fill_list = [value for value in fill_list if pd.notna(value)]
fill_list

['śląskie',
 'opolskie',
 'kujawsko-pomorskie',
 'dolnośląskie',
 'podkarpackie',
 'warmińsko-mazurskie',
 'wielkopolskie',
 'pomorskie',
 'lubelskie',
 'łódzkie',
 'mazowieckie',
 'zachodniopomorskie',
 'małopolskie',
 'podlaskie',
 'świętokrzyskie',
 'lubuskie',
 'Małopolskie',
 'Zachodniopomorskie',
 'Mazowieckie',
 'Lubuskie',
 'Dolnośląskie',
 'Wielkopolskie',
 'LubelskieWarmińsko-MazurskiePokaż dodatkowe (1)',
 'ŚwiętokrzyskieKujawsko-PomorskiePokaż dodatkowe (1)',
 'Pomorskie',
 'Łódzkie',
 'Świętokrzyskie',
 'Podlaskie',
 'Kujawsko-PomorskiePomorskiePokaż dodatkowe (1)',
 'Śląskie',
 'Lubelskie',
 'Podkarpackie',
 'MazowieckiePodkarpackiePokaż dodatkowe (1)',
 'Kujawsko-Pomorskie',
 'WielkopolskiePomorskiePokaż dodatkowe (1)',
 'Opolskie',
 'Warmińsko-Mazurskie',
 'PodkarpackieMazowieckieWielkopolskiePokaż dodatkowe (2)',
 'ŚwiętokrzyskieMałopolskieOpolskiePokaż dodatkowe (2)',
 'MazowieckieDolnośląskiePokaż dodatkowe (1)',
 'WielkopolskieLubuskiePokaż dodatkowe (1)',
 'Mazowie

In [105]:
mapping = {
    'lubuskie': ['lubuskie'],
    'pomorski': ['pomorskie'],
    'mazowieckie': ['mazowieckie'],
    'wielkopolska, kujawsko pomorskie': ['kujawsko-pomorskie'],  # Example of focusing on one voivodeship
    'łódzkie': ['łódzkie'],
    'małopolska, cala polska': ['małopolskie'],  # Example of focusing on one voivodeship
    'kujawsko pomorskie, pomorskie, lubuskie, wielkopolskie zach pomorskie  250 km': ['kujawsko-pomorskie'],  # Example
    'tylko w obrebie 70km od jelenie gory': fill_list,
    'Polska': fill_list,
    'świętokrzyskie - Radków': ['świętokrzyskie'],
    '100 km od radomia': fill_list,
    'pomorskie ': ['pomorskie'],
    '100km max bielski żywiecki oświęcimski 100km max ': fill_list,
    'max 30km od Krakowa': fill_list,
    'lubuskie, dolnośląskie': ['lubuskie'],  # Example of focusing on one voivodeship
    'najlepiej wielkopolska ale moze byc dalej': ['wielkopolskie'],
    'zielonogorski swiebodzinski ': fill_list,
    'małopolskie, śląskie': ['małopolskie'],  # Example of focusing on one voivodeship
    'tylko łódzkie ': ['łódzkie'],
    'wielkopolskie': ['wielkopolskie'],
    'max katowice': fill_list,
    'cala polska': fill_list,
    'zach-pomorskie': ['zachodniopomorskie'],
}


In [106]:
# Map the 'query' values using the mapping dictionary
query_df['obszar działalności'] = query_df['obszar działalności'].map(mapping)


In [107]:
query_df.at[8, 'obszar działalności'] = fill_list

In [108]:
query_df

,Firma,Imie i nazwisko,Tel,email,www,data kontkatu,Free raport sent?,typy dzialalnosci,obszar działalności,feedback call,paying?,notes,kategorie colab,enhanced opis gpt\n(na podstawie strony)
0,Lenpol,Krzysztof Czepukowicz,605895476,info@lenpol.pl,lenpol.pl,15/07/2024,NaN,"wyburzenia, rozbiórki",[lubuskie],17/07 - nie odbiera,NaN,NaN,rozbiórki wyburzenia,Firma specjalizuje się w wyburzeniach i rozbió...
1,Nowy Dach FHU,Roman Mejer,511016044,nowydach27@wp.pl,http://www.nowy-dach.com.pl/,15/07/2024,NaN,"pokrycia dachowe, hurtownia",[pomorskie],blisko nas szkoły itd. bez procedury nieraz si...,NaN,NaN,dekartstwo\nmateriały prefabrykaty budowlane,"Firma z siedzibą w Nowej Karczmie, specjalizuj..."
2,Gut-Bud,Magdalena,606 663 308,gut-bud@wp.pl,https://gut-bud.pl/,15/07/2024,NaN,"elewacje, budowy domów",[mazowieckie],17/07 - nie odbiera,NaN,NaN,budownictwo - obiekty wykończenia\nbudownictwo...,Specjalizuje się w budowie domów oraz pracach ...
3,Grinbud,Dominik Sieszchuła,663 007 302,biuro@grinbud.pl,https://www.grinbud.pl/,15/07/2024,NaN,roboty budowlane,[kujawsko-pomorskie],16/07 - on ma dostep do biznespolska i tam wsz...,not interested,NaN,budownictwo - wielobranżowe/specjalistyczne\n,Firma zajmuje się szerokim zakresem robót budo...
4,Montplast,Iwona,696 492 894,biuro@montplast.pl,https://www.montplast.com.pl/,15/07/2024,NaN,tylko wymiana okien i drzwi,[łódzkie],pani jest tylko od tego i analizuje maja bizne...,NaN,"Iwona - jedzie na urlop nad moze, od piatku je...",okna/drzwi/bramy/ogrodzenia\n\n\n\n\n\n,Specjalizuje się w wymianie okien i drzwi. Fir...
5,Jati,Jacek Kaczor,503903511,jatimeble@gmail.com,https://jati.pl/,12/07/2024,NaN,"lady recepcyjne, hotele, stolarka na wymiar",[małopolskie],wyslac wiecej,NaN,NaN,Meble/wyposażenie wnętrz\nStolarka/drewno\nPro...,Jati Sp. z o.o. specjalizuje się w kompleksowy...
6,EkoWodrol,Adrianna Wiśniewska - przetargi,693580282,adrianna.wisniewska@ekowodrol.eu,https://ekowodrol.pl/,12/07/2024,NaN,sieci wodociagowo kanalizacyjne,[kujawsko-pomorskie],nieodbiera - 17/07,NaN,NaN,Budownictwo - wodno-kanalizacyjne\nAutomatyka ...,EkoWodrol Sp. z o.o. specjalizuje się w komple...
7,Jadar SC,"Jarosław Niedzielski , Adam Balicki",502 179 911,jadarsc@gmail.com,http://www.jadar.eu/,12/07/2024,NaN,sieci wodno kanalziacyjne na koparki,"[śląskie, opolskie, kujawsko-pomorskie, dolnoś...",yes,yes,NaN,Budownictwo - sanitarno-hydrauliczne\nKlimatyz...,Instalacje Iwanicki specjalizuje się w instala...
8,ISO Iwanicki,Paweł Iwanicki,602 559 627,instalacje@op.pl,https://www.instalacjeiwanicki.pl/,12/07/2024,NaN,instalacje hale wieskze miekszje kanalizacja,"[śląskie, opolskie, kujawsko-pomorskie, dolnoś...",ma zobaczyc dzisiaj - 16/07,NaN,NaN,Budownictwo - wodno-kanalizacyjne\nUsługi sprz...,Jadar specjalizuje się w budowie sieci wodno-k...
9,Wodner,Kazimierz Żurawski,607 994 355,wodner@interia.pl,NaN,12/07/2024,NaN,NaN,[łódzkie],ma rzucic okiem - przycisnij -16/07,NaN,NaN,NaN,NaN


In [12]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv()
api_token = os.getenv("TENDER_API_KEY")
api_url = 'http://127.0.0.1:8000/search'

query = '''
MarkX	Piotr Rajshttps://www.markx-rajs.pl/ budownictow drogowe; Drogi, nasypy stabilizacje kanalizacje, ale tez duzo innych- patrz strona. Lokalizacja: cala polska
kanalizacja kanalizacja kanalizacja
'''

cats = [
    "budownictwo - drogi chodniki place",
    "budownictwo - wodno-kanalizacyjne",
    "budownictwo - sanitarno-hydrauliczne",
]

location = ['pomorskie']

# Define the request payload
payload = {
    "query": query,
    "categories": cats,
    "locations": location
}

print("Payload:", payload)

# Define the headers
headers = {
    'Content-Type': 'application/json',
}

# Make the POST request
try:
    response = requests.post(api_url, headers=headers, data=json.dumps(payload), timeout=1000000)
    
    # Check the response status and print the response data
    if response.status_code == 200:
        print("Success!")
        print("Response data:", response.json())
    else:
        print("Failed!")
        print("Status code:", response.status_code)
        print(json.loads(response.text))
except Exception as e:
    print(f"An error occurred: {e}")
pd.DataFrame(data=json.loads(response.text))

Payload: {'query': '\nMarkX\tPiotr Rajshttps://www.markx-rajs.pl/ budownictow drogowe; Drogi, nasypy stabilizacje kanalizacje, ale tez duzo innych- patrz strona. Lokalizacja: cala polska\nkanalizacja kanalizacja kanalizacja\n', 'categories': ['budownictwo - drogi chodniki place', 'budownictwo - wodno-kanalizacyjne', 'budownictwo - sanitarno-hydrauliczne'], 'locations': ['pomorskie']}
Success!
Response data: [{'id': '533464b0-04e0-4070-98e8-d3ff4bb2c269', 'summary': 'Zimowe utrzymanie dróg i chodników na terenie Gminy Wejherowo w podziale na cztery części. Część zamówienia: LOT-0001. Tytuł: Zimowe utrzymanie dróg i chodników na terenie Gminy Wejherowo - Część 1. Opis: Nazwa drogi/ długość odcinka do utrzymanie w km 1 Ulica Drzewiarza w Gościcinie od drogi krajowej nr 6 do pętli autobusowej. 1,50 2 Ulica Słoneczna w Gościcinie od skrzyżowania z drogą nr 6 wraz pętlą autobusową (zwrócić szczególna uwagę na prawidłowe utrzymanie przejazdu pod wiaduktem kolejowym) 2,00 3 Ulica Robakowska w 

,id,summary,opis,Województwo,Miasto,url,Zamawiający,Data Zakończenia,Kategoria
0,533464b0-04e0-4070-98e8-d3ff4bb2c269,Zimowe utrzymanie dróg i chodników na terenie ...,Zimowe utrzymanie dróg i chodników na terenie ...,[pomorskie],wejherowski,https://www.oferent.com.pl/przetargi/zimowe-ut...,Gmina Wejherowo,2024-09-10,budownictwo - drogi chodniki place
1,2a9852dd-9459-48f3-ac2f-05d0cd68c93b,Przebudowa drogi powiatowej 1732G Prądzona - L...,Przebudowa drogi powiatowej 1732G Prądzona - L...,[pomorskie],bytowski,https://www.oferent.com.pl/przetargi/przebudow...,Zarząd Dróg Powiatowych w Bytowie,2024-08-06,budownictwo - drogi chodniki place
2,08d47876-a097-4cb2-a82e-acd781e78500,Budowa drogi leśnej nr 171 na terenie leśnictw...,Budowa drogi leśnej nr 171 na terenie leśnictw...,[pomorskie],słupski,https://www.oferent.com.pl/przetargi/budowa-dr...,Nadleśnictwo Ustka,2024-08-12,budownictwo - drogi chodniki place
3,c87cc68c-f89b-4eb9-9834-37ff40645ef0,Utwardzenie drogi gruntowej Frank - Łążek. Kró...,Utwardzenie drogi gruntowej Frank - ŁążekKrótk...,[pomorskie],starogardzki,https://www.oferent.com.pl/przetargi/utwardzen...,Gmina Kaliska,2024-08-09,budownictwo - drogi chodniki place
4,a663d0ba-1cc5-4029-a4bc-bdcc4e750556,Rozbudowa skrzyżowania drogi krajowej nr 20 z ...,",,Rozbudowa skrzyżowania drogi krajowej nr 20 ...",[pomorskie],Gdańsk,https://www.oferent.com.pl/przetargi/rozbudowa...,Generalna Dyrekcja Dróg Krajowych i Autostrad ...,2024-08-08,budownictwo - drogi chodniki place


In [1]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()
api_token = os.getenv("TENDER_API_KEY")
api_url = os.getenv("TENDER_URL")

query = '''
MarkX	Piotr Rajshttps://www.markx-rajs.pl/ budownictow drogowe; Drogi, nasypy stabilizacje kanalizacje, ale tez duzo innych- patrz strona. Lokalizacja: cala polska
'''

cats = [
    "budownictwo - drogi chodniki place",
    "budownictwo - wodno-kanalizacyjne",
    "budownictwo - sanitarno-hydrauliczne",
]

location = ['pomorskie']

# Define the request payload
payload = {
    "query": query,
    "categories": cats,
    "locations": location
}

print("Payload:", payload)

# Define the headers
headers = {
    'Content-Type': 'application/json',
    'x-api-key': api_token
}

# Make the POST request
try:
    response = requests.post(api_url, headers=headers, data=json.dumps(payload), timeout=1000000)
    
    # Check the response status and print the response data
    if response.status_code == 200:
        print("Success!")
        print("Response data:", response.json())
    else:
        print("Failed!")
        print("Status code:", response.status_code)
        print(json.loads(response.text))
except Exception as e:
    print(f"An error occurred: {e}")

/Users/krystianpietrzak/Documents/ML/Tender-AI/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Payload: {'query': '\nMarkX\tPiotr Rajshttps://www.markx-rajs.pl/ budownictow drogowe; Drogi, nasypy stabilizacje kanalizacje, ale tez duzo innych- patrz strona. Lokalizacja: cala polska\n', 'categories': ['budownictwo - drogi chodniki place', 'budownictwo - wodno-kanalizacyjne', 'budownictwo - sanitarno-hydrauliczne'], 'locations': ['pomorskie']}
Success!
Response data: [{'id': '533464b0-04e0-4070-98e8-d3ff4bb2c269', 'summary': 'Zimowe utrzymanie dróg i chodników na terenie ...', 'opis': 'Zimowe utrzymanie dróg i chodników na terenie Gminy WejherowoZimowe utrzymanie dróg i chodników na terenie Gminy Wejherowo w podziale na cztery częściCzęść zamówienia: LOT-0001Tytuł: Zimowe utrzymanie dróg i chodników na terenie Gminy Wejherowo - Część 1Opis: Nazwa drogi/ długość odcinka do utrzymanie w km 1 Ulica Drzewiarza w Gościcinie od drogi krajowej nr 6 do pętli autobusowej. 1,50 2 Ulica Słoneczna w Gościcinie od skrzyżowania z drogą nr 6 wraz pętlą autobusową (zwrócić szczególna uwagę na praw

In [43]:
df = pd.read_csv('mnt/efs/data_pre_v2.csv')
df = df.dropna(subset='category')

In [124]:
for i in range(9):
    try:
        cats = query_df.iloc[i]['kategorie colab'].split('\n')
        cats = [cat.strip().lower() for cat in cats]
        
        import requests
        import json

        # Define the API Gateway URL
        api_url = "http://localhost:8000/search"

        # Define the request payload
        payload = {
            "query": query_df.iloc[i]['enhanced opis gpt\n(na podstawie strony)'],
            "categories": cats,
            "locations": query_df.iloc[i]['obszar działalności']
        }
        print(payload)
        # Define the headers
        headers = {
            'Content-Type': 'application/json'
        }

        # Make the POST request
        response = requests.post(api_url, headers=headers, data=json.dumps(payload), timeout=1000000)

        # Check the response status and print the response data
        if response.status_code == 200:
            print("Success!")
            print("Response data:", response.json())
        else:
            print("Failed!")
            print("Status code:", response.status_code)
            print("Response data:", response.text)

        data = {
            'url': [query_df.iloc[i]['Firma']] * len(json.loads(response.text)),
            'matches' : [result[0] for result in json.loads(response.text)]

        }
        df_result = pd.DataFrame(data)
        import os
        if os.path.exists('results.csv'):
            df_ = pd.read_csv('results.csv')
            df_result = pd.concat([df_, df_result])
        df_result.to_csv('results.csv', index=False)
    except:
        pass
    

{'query': 'Firma specjalizuje się w wyburzeniach i rozbiórkach budynków. Ich oferta obejmuje kompleksowe usługi związane z demontażem i usuwaniem konstrukcji budowlanych. Poszukują przetargów związanych z wyburzeniami budynków, usuwaniem gruzu oraz przygotowaniem terenu pod nowe inwestycje.', 'categories': ['rozbiórki wyburzenia'], 'locations': ['lubuskie']}
Failed!
Status code: 500
Response data: Internal Server Error
{'query': 'Firma z siedzibą w Nowej Karczmie, specjalizująca się w usługach dekarskich na terenie województwa pomorskiego. Oferują szeroki wachlarz usług, w tym budowę dachów, remonty, sprzedaż materiałów dekarskich (np. blachodachówki, dachówki, gonty bitumiczne) oraz montaż orynnowania i okien dachowych. Firma poszukuje przetargów na budowę i remonty dachów, a także na dostawy materiałów budowlanych związanych z pokryciami dachowymi\u200b\u200b\u200b\u200b\u200b\u200b.\n\n', 'categories': ['dekartstwo', 'materiały prefabrykaty budowlane'], 'locations': ['pomorskie']}
F

In [1]:
df_right = pd.read_csv('mnt/efs/data_pre_v2.csv')


NameError: name 'pd' is not defined

In [113]:
df_final = pd.merge(left=df_result, right=df_right, left_on='matches', right_on='id')


In [121]:
df_final[['url','id', 'category','Krótki opis', 'Województwo',]].to_csv('results_matched.csv')

In [72]:
json.loads(response.text)[0]

{'id': None,
 'metadata': {'id': 6262},
 'page_content': 'Poprawa bezpieczeństwa pieszych i rowerzystów poprzez budowę i przebudowę infrastruktury drogowej na sieci dróg wojewódzkich w województwie pomorskim w podziale na 5 części.',
 'type': 'Document'}